# YouTube to MP3 Batch Converter for Google Colab 🎵

This notebook allows you to batch convert YouTube videos to MP3 files by uploading a CSV file containing YouTube links.

## Features:
- Upload CSV file with YouTube links
- Batch download and convert to MP3
- Real-time progress tracking
- Automatic zip file creation for easy download

## CSV Format:
Your CSV file should have this structure:
```csv
LINKS
https://www.youtube.com/watch?v=dQw4w9WgXcQ
https://youtu.be/jNQXAC9IVRw
```

Row 1 must contain the header `LINKS` (case-insensitive). YouTube URLs start from Row 2.


## Step 1: Install Dependencies


In [ ]:
# Install required packages
!pip install -U yt-dlp pandas -q

print("✅ Dependencies installed")


## Step 2: Upload Your CSV File


In [ ]:
from google.colab import files
import os

# Upload CSV file
uploaded = files.upload()

# Get the uploaded file name
csv_file = None
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        csv_file = filename
        print(f"✅ CSV file uploaded: {csv_file}")
        break

if not csv_file:
    print("❌ No CSV file found in upload")


## Step 3: Import the Converter Script


In [ ]:
# Import the converter functions
import sys
import os
import csv
import subprocess
import time
from pathlib import Path
from typing import List, Dict, Tuple

# Configuration
OUTPUT_DIR = "/content/downloads"
TEMP_DIR = "/content/temp"
AUDIO_QUALITY = "192K"

def setup_directories():
    """Create necessary directories if they don't exist"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(TEMP_DIR, exist_ok=True)
    print(f"✅ Directories created: {OUTPUT_DIR}, {TEMP_DIR}")

def check_ytdlp():
    """Check if yt-dlp is installed"""
    try:
        result = subprocess.run(
            ["yt-dlp", "--version"],
            capture_output=True,
            text=True,
            timeout=10
        )
        if result.returncode == 0:
            print(f"✅ yt-dlp installed: {result.stdout.strip()}")
            return True
    except (subprocess.TimeoutExpired, FileNotFoundError):
        pass
    
    print("❌ yt-dlp not found. Installing...")
    try:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
            check=True,
            timeout=120
        )
        print("✅ yt-dlp installed successfully")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install yt-dlp: {e}")
        return False

def parse_csv(csv_path: str) -> List[Dict[str, str]]:
    """Parse CSV file and extract YouTube links"""
    links = []
    
    try:
        import pandas as pd
        df = pd.read_csv(csv_path)
        
        # Find LINKS column (case-insensitive)
        link_column = None
        for col in df.columns:
            if col.upper() == 'LINKS':
                link_column = col
                break
        
        if not link_column:
            raise ValueError("No 'LINKS' column found in CSV")
        
        # Extract links
        row_number = 2
        for idx, row in df.iterrows():
            link = str(row[link_column]).strip()
            if link and ('youtube.com' in link or 'youtu.be' in link):
                links.append({
                    'row': row_number,
                    'link': link
                })
            row_number += 1
        
        print(f"✅ Found {len(links)} YouTube links in CSV")
        return links
        
    except Exception as e:
        print(f"❌ Error parsing CSV: {e}")
        raise

def get_video_title(url: str) -> str:
    """Get video title from YouTube URL"""
    try:
        result = subprocess.run(
            ["yt-dlp", "--get-title", url],
            capture_output=True,
            text=True,
            timeout=30
        )
        if result.returncode == 0:
            title = result.stdout.strip()
            title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_'))
            title = title.replace(' ', '_')[:50]
            return title
    except Exception as e:
        pass
    
    return f"audio_{int(time.time())}"

def download_video_to_mp3(url: str, output_dir: str, row_number: int) -> Tuple[bool, str]:
    """Download YouTube video and convert to MP3"""
    try:
        print(f"\n📥 [{row_number}] Downloading: {url}")
        
        # Get video title
        title = get_video_title(url)
        final_output = os.path.join(output_dir, f"{title}.mp3")
        
        # yt-dlp command
        cmd = [
            "yt-dlp",
            "-x",
            "--audio-format", "mp3",
            "--audio-quality", AUDIO_QUALITY,
            "-o", final_output,
            "--no-playlist",
            "--no-warnings",
            url
        ]
        
        # Run yt-dlp
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Monitor progress
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output and '%' in output:
                print(f"\r⏳ [{row_number}] {output.strip()}", end='', flush=True)
        
        return_code = process.poll()
        
        if return_code == 0:
            # Check if file was created
            if os.path.exists(final_output):
                file_size = os.path.getsize(final_output) / (1024 * 1024)
                print(f"\n✅ [{row_number}] Download complete: {title}.mp3 ({file_size:.2f} MB)")
                return True, f"Downloaded: {title}.mp3"
            else:
                # Check for files with similar names
                base_name = os.path.splitext(final_output)[0]
                for file in os.listdir(output_dir):
                    if file.startswith(os.path.basename(base_name)):
                        actual_path = os.path.join(output_dir, file)
                        if not file.endswith('.mp3'):
                            new_path = os.path.splitext(actual_path)[0] + '.mp3'
                            os.rename(actual_path, new_path)
                            file_size = os.path.getsize(new_path) / (1024 * 1024)
                            print(f"\n✅ [{row_number}] Download complete: {os.path.basename(new_path)} ({file_size:.2f} MB)")
                            return True, f"Downloaded: {os.path.basename(new_path)}"
                        else:
                            file_size = os.path.getsize(actual_path) / (1024 * 1024)
                            print(f"\n✅ [{row_number}] Download complete: {os.path.basename(actual_path)} ({file_size:.2f} MB)")
                            return True, f"Downloaded: {os.path.basename(actual_path)}"
                
                return False, "File was not created"
        else:
            error = process.stderr.read()
            print(f"\n❌ [{row_number}] Download failed: {error}")
            return False, f"Error: {error[:100]}"
            
    except Exception as e:
        print(f"\n❌ [{row_number}] Error: {e}")
        return False, str(e)

print("✅ Converter functions loaded")


## Step 4: Process Your CSV File


In [ ]:
if csv_file:
    print("=" * 60)
    print("YouTube to MP3 Batch Converter")
    print("=" * 60)
    
    # Setup
    setup_directories()
    
    if not check_ytdlp():
        print("❌ Cannot proceed without yt-dlp")
    else:
        # Parse CSV
        try:
            links = parse_csv(csv_file)
        except Exception as e:
            print(f"❌ Failed to parse CSV: {e}")
            links = []
        
        if links:
            print(f"\n🚀 Starting batch download of {len(links)} videos...\n")
            
            results = {
                'total': len(links),
                'success': 0,
                'failed': 0,
                'failed_links': []
            }
            
            for i, item in enumerate(links, 1):
                success, message = download_video_to_mp3(
                    item['link'],
                    OUTPUT_DIR,
                    item['row']
                )
                
                if success:
                    results['success'] += 1
                else:
                    results['failed'] += 1
                    results['failed_links'].append({
                        'row': item['row'],
                        'link': item['link'],
                        'error': message
                    })
                
                # Delay between downloads (1 second)
                if i < len(links):
                    time.sleep(1)
            
            # Print summary
            print("\n" + "=" * 60)
            print("📊 Processing Complete")
            print("=" * 60)
            print(f"Total: {results['total']}")
            print(f"✅ Success: {results['success']}")
            print(f"❌ Failed: {results['failed']}")
            print(f"📁 Output directory: {OUTPUT_DIR}")
            
            if results['failed_links']:
                print("\n⚠️  Failed Downloads:")
                for item in results['failed_links']:
                    print(f"  Row {item['row']}: {item['link']}")
                    print(f"    Error: {item['error']}")
else:
    print("❌ Please upload a CSV file first")


## Step 5: Download Your MP3 Files


In [ ]:
# Create a zip file of all MP3s for easy download
import zipfile

if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
    zip_path = os.path.join(OUTPUT_DIR, "all_mp3s.zip")
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        mp3_count = 0
        for file in os.listdir(OUTPUT_DIR):
            if file.endswith('.mp3'):
                zipf.write(
                    os.path.join(OUTPUT_DIR, file),
                    file
                )
                mp3_count += 1
    
    if mp3_count > 0:
        print(f"✅ Created zip file with {mp3_count} MP3 files: {zip_path}")
        print("\n📥 Download the zip file:")
        files.download(zip_path)
    else:
        print("⚠️  No MP3 files found to zip")
else:
    print("⚠️  No files to download")


## Alternative: Download Individual Files

If you prefer to download files individually instead of a zip:


In [ ]:
# Download individual MP3 files
if os.path.exists(OUTPUT_DIR):
    mp3_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.mp3')]
    
    if mp3_files:
        print(f"Found {len(mp3_files)} MP3 files. Downloading...")
        for file in mp3_files:
            file_path = os.path.join(OUTPUT_DIR, file)
            files.download(file_path)
        print("✅ All files downloaded")
    else:
        print("⚠️  No MP3 files found")
else:
    print("⚠️  Output directory not found")
